# Simple UBI: How taxcalc uses the baseline for all binning

This implements a basic $10,000 UBI. It demonstrates how taxcalc uses the counts in the baseline calculator for difference tables and each's distribution table. Therefore results are not mirror images of each other when comparing two policies.

*Data: CPS  |  Tax year: 2018  |  Type: Static  |  Author: Max Ghenis  |  Date run: 2018-02-21*

## Imports

In [1]:
import taxcalc as tc
import matplotlib as mpl
import pandas as pd
import numpy as np
from bokeh.io import show, output_notebook
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
print tc.__version__

0.16.1


## Setup

Use publicly-available CPS input file.

In [3]:
recs = tc.Records.cps_constructor()

Specify `Calculator` object for static analysis of current-law (TCJA) policy.

In [4]:
pol = tc.Policy()
base_calc = tc.Calculator(policy=pol, records=recs)

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [5]:
base_calc.advance_to_year(2018)
base_calc.calc_all()

Create non-taxable UBI reform.

In [6]:
ubi_amount = 10000
ubi_reform = {2018: {'_UBI_u18': [ubi_amount], 
                     '_UBI_1820': [ubi_amount], 
                     '_UBI_21': [ubi_amount],
                     '_UBI_ecrt': [1.0]}}

Implement reform and check for reform error messages.

In [7]:
pol.implement_reform(ubi_reform)

## Calculate

Specify Calculator object for static analysis of reform policy.

In [8]:
ubi_calc = tc.Calculator(policy=pol, records=recs)
ubi_calc.advance_to_year(2018)
ubi_calc.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


## Results

Calculate distribution and difference tables using the UBI policy as both reform and baseline, and using deciles and income bins.

### Distribution tables

#### By income bin

In [9]:
base_dist_bin, ubi_dist_bin = base_calc.distribution_tables(ubi_calc, groupby='webapp_income_bins')
ubi_dist_bin_rev, base_dist_bin_rev = ubi_calc.distribution_tables(base_calc, groupby='webapp_income_bins')

Compare baseline calcs.

In [10]:
base_dist_bin

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"1,434,322","-21,830,748,823",0,"23,703,387,741",60,"7,771,021",0,0,0,"-21,837,914,944",...,0,0,0,0,"101,081,338","-101,081,338","455,262,554","354,181,216","-26,081,112,653","-26,435,293,869"
1,"12,505,153","38,860,950,804","10,640,149","121,200,769,834","10,905","81,488,552",0,"960,742,611","26,516,714","38,784,575,763",...,"723,851","27,240,565","14,677,375",0,"4,318,987,039","-4,306,423,849","5,485,487,576","1,179,063,727","58,661,744,841","57,482,681,114"
2,"15,293,225","134,385,206,367","11,809,689","205,185,159,100","245,851","4,593,216,046",0,"15,931,900,208","1,463,103,665","130,603,370,581",...,"17,517,018","1,480,620,683","401,334,364",0,"14,307,348,705","-13,228,062,387","18,648,969,477","5,420,907,090","235,975,389,764","230,554,482,674"
3,"20,287,706","285,279,409,377","16,021,434","287,553,535,444","601,566","11,180,337,910",0,"93,852,844,285","9,316,419,837","276,040,626,925",...,"36,596,656","9,353,016,493","2,346,920,468",0,"20,614,472,508","-13,608,376,484","40,463,284,764","26,854,908,280","508,439,371,745","481,584,463,465"
4,"19,250,236","408,329,443,156","16,091,829","289,947,808,545","829,274","16,021,147,927",0,"184,713,326,528","19,389,208,124","395,442,357,432",...,"57,480,247","19,446,688,370","4,911,353,829",0,"18,529,776,827","-3,994,442,285","57,742,783,385","53,748,341,100","671,531,486,298","617,783,145,198"
5,"14,981,032","432,899,346,714","12,622,917","238,928,784,077","975,141","19,841,770,677",0,"230,186,571,533","24,560,693,227","417,602,187,813",...,"60,797,161","24,621,490,388","6,185,345,094",0,"12,332,114,906","6,104,030,389","60,821,320,041","66,925,350,430","671,416,887,548","604,491,537,117"
6,"27,918,245","1,177,276,188,149","22,697,199","462,126,145,753","3,384,216","72,213,301,383",0,"726,692,132,330","83,726,443,224","1,123,984,379,118",...,"126,199,854","83,852,643,078","17,665,477,359",0,"13,748,648,498","52,438,517,220","162,047,234,346","214,485,751,566","1,714,799,013,860","1,500,313,262,294"
7,"18,307,252","1,159,900,237,500","13,521,837","298,751,768,642","3,832,942","92,038,908,924",0,"807,115,714,254","101,897,791,642","1,094,538,220,494",...,"125,996,124","102,023,787,765","16,679,330,353",0,"4,518,450,885","80,826,006,528","158,019,013,239","238,845,019,766","1,585,359,332,521","1,346,514,312,755"
8,"29,309,573","3,158,410,292,414","16,834,870","398,750,804,536","11,412,037","316,538,625,163",0,"2,471,779,362,951","357,912,109,960","2,944,712,691,341",...,"149,341,121","358,061,451,081","35,749,586,464",0,"3,500,419,042","318,811,445,575","421,059,647,604","739,871,093,179","4,028,819,923,722","3,288,948,830,543"
9,"9,503,573","2,174,244,835,363","3,575,020","87,408,937,190","5,684,303","182,447,760,754",0,"1,901,841,947,145","354,548,207,113","2,047,795,134,234",...,"290,722,050","354,838,929,164","12,777,006,151","1,268,944,185","751,909,620","342,578,957,579","226,620,742,327","569,199,699,905","2,601,580,784,581","2,032,381,084,676"


In [11]:
base_dist_bin_rev

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"110,844","-20,964,853,729",0,"1,882,518,336",60,"7,771,021",0,0,0,"-20,972,019,850",...,0,0,0,0,"26,282,736","-26,282,736","199,978,905","173,696,169","-25,591,068,862","-25,764,765,031"
1,"40,803","-356,100,086",0,"483,900,672",0,0,0,0,0,"-356,100,086",...,0,0,0,0,"7,893,699","-7,893,699","50,476,050","42,582,351","-220,823,311","-263,405,662"
2,"11,163,888","32,911,139,289","9,136,486","91,348,663,810","10,905","81,488,552",0,"952,971,223","25,911,104","32,834,764,248",...,"405,183","26,316,286","14,105,164",0,"3,258,884,791","-3,246,673,669","4,732,203,581","1,485,529,912","48,347,224,151","46,861,694,239"
3,"13,099,479","103,048,007,398","9,716,714","165,913,534,691","217,819","4,052,730,207",0,"14,314,381,088","1,308,402,254","99,734,162,907",...,"13,683,987","1,322,086,241","247,602,652",0,"7,861,611,339","-6,787,127,750","14,245,875,898","7,458,748,148","183,006,047,547","175,547,299,399"
4,"17,062,882","209,892,996,266","12,933,918","224,427,219,344","512,571","9,267,255,596",0,"78,917,523,426","7,835,773,077","202,264,409,235",...,"35,068,744","7,870,841,821","1,132,603,342",0,"9,625,054,339","-2,886,815,860","29,434,896,806","26,548,080,946","389,284,119,067","362,736,038,121"
5,"16,426,264","308,133,539,048","13,399,370","228,613,974,343","708,277","12,918,147,525",0,"145,308,710,978","15,384,409,414","297,874,695,246",...,"44,439,754","15,428,849,169","2,246,005,407",0,"9,623,768,249","3,559,075,513","43,201,817,396","46,760,892,909","511,106,395,876","464,345,502,967"
6,"32,245,650","934,370,371,811","26,462,074","504,985,007,738","2,576,961","49,741,642,650",0,"517,481,951,945","57,357,452,379","897,078,916,702",...,"167,054,366","57,524,506,746","8,455,833,007",0,"24,435,002,992","24,633,670,747","129,958,917,491","154,592,588,237","1,427,743,884,971","1,273,151,296,733"
7,"23,253,128","1,033,360,877,365","18,687,850","400,277,292,442","3,108,441","68,016,158,081",0,"636,956,189,171","79,458,890,462","984,343,757,059",...,"115,624,242","79,574,514,703","12,110,585,410",0,"19,459,290,885","48,004,638,409","140,331,023,448","188,335,661,857","1,503,770,525,128","1,315,434,863,271"
8,"41,630,877","3,538,453,845,943","28,108,545","663,440,283,642","11,802,357","317,086,841,717",0,"2,617,882,984,037","354,639,365,848","3,320,930,454,641",...,"174,226,745","354,813,592,592","50,022,084,138",0,"16,825,905,328","287,965,603,127","478,304,663,978","766,270,267,105","4,640,593,646,286","3,874,323,379,181"
9,"13,618,878","2,751,636,533,926","5,331,880","131,254,252,528","7,942,273","250,200,391,353",0,"2,368,720,855,161","425,449,380,338","2,579,294,747,573",...,"304,009,361","425,753,389,699","22,445,769,771","1,179,894,187","1,576,754,424","402,910,759,691","306,760,079,281","709,670,838,971","3,305,986,803,198","2,596,315,964,227"


Compare UBI distributions.

In [12]:
ubi_dist_bin

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"1,434,322","-21,830,748,823",0,"23,703,387,741",60,"7,771,021",0,0,0,"-21,837,914,944",...,0,0,0,0,"101,081,338","-101,081,338","455,262,554","354,181,216","217,934,047","-136,247,169"
1,"12,505,153","38,860,950,804","10,640,149","121,200,769,834","10,905","81,488,552",0,"960,742,611","26,516,714","38,784,575,763",...,"723,851","27,240,565","14,677,375",0,"4,318,987,039","-4,306,423,849","5,485,487,576","1,179,063,727","214,820,640,041","213,641,576,314"
2,"15,293,225","134,385,206,367","11,809,689","205,185,159,100","245,851","4,593,216,046",0,"15,931,900,208","1,463,103,665","130,603,370,581",...,"17,517,018","1,480,620,683","401,334,364",0,"14,307,348,705","-13,228,062,387","18,648,969,477","5,420,907,090","448,177,175,464","442,756,268,374"
3,"20,287,706","285,279,409,377","16,021,434","287,553,535,444","601,566","11,180,337,910",0,"93,852,844,285","9,316,419,837","276,040,626,925",...,"36,596,656","9,353,016,493","2,346,920,468",0,"20,614,472,508","-13,608,376,484","40,463,284,764","26,854,908,280","816,421,277,745","789,566,369,465"
4,"19,250,236","408,329,443,156","16,091,829","289,947,808,545","829,274","16,021,147,927",0,"184,713,326,528","19,389,208,124","395,442,357,432",...,"57,480,247","19,446,688,370","4,911,353,829",0,"18,529,776,827","-3,994,442,285","57,742,783,385","53,748,341,100","995,800,886,698","942,052,545,598"
5,"14,981,032","432,899,346,714","12,622,917","238,928,784,077","975,141","19,841,770,677",0,"230,186,571,533","24,560,693,227","417,602,187,813",...,"60,797,161","24,621,490,388","6,185,345,094",0,"12,332,114,906","6,104,030,389","60,821,320,041","66,925,350,430","953,584,827,848","886,659,477,417"
6,"27,918,245","1,177,276,188,149","22,697,199","462,126,145,753","3,384,216","72,213,301,383",0,"726,692,132,330","83,726,443,224","1,123,984,379,118",...,"126,199,854","83,852,643,078","17,665,477,359",0,"13,748,648,498","52,438,517,220","162,047,234,346","214,485,751,566","2,293,240,956,060","2,078,755,204,494"
7,"18,307,252","1,159,900,237,500","13,521,837","298,751,768,642","3,832,942","92,038,908,924",0,"807,115,714,254","101,897,791,642","1,094,538,220,494",...,"125,996,124","102,023,787,765","16,679,330,353",0,"4,518,450,885","80,826,006,528","158,019,013,239","238,845,019,766","2,003,621,503,921","1,764,776,484,155"
8,"29,309,573","3,158,410,292,414","16,834,870","398,750,804,536","11,412,037","316,538,625,163",0,"2,471,779,362,951","357,912,109,960","2,944,712,691,341",...,"149,341,121","358,061,451,081","35,749,586,464",0,"3,500,419,042","318,811,445,575","421,059,647,604","739,871,093,179","4,779,643,319,422","4,039,772,226,243"
9,"9,503,573","2,174,244,835,363","3,575,020","87,408,937,190","5,684,303","182,447,760,754",0,"1,901,841,947,145","354,548,207,113","2,047,795,134,234",...,"290,722,050","354,838,929,164","12,777,006,151","1,268,944,185","751,909,620","342,578,957,579","226,620,742,327","569,199,699,905","2,861,699,893,581","2,292,500,193,676"


In [13]:
ubi_dist_bin_rev

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"110,844","-20,964,853,729",0,"1,882,518,336",60,"7,771,021",0,0,0,"-20,972,019,850",...,0,0,0,0,"26,282,736","-26,282,736","199,978,905","173,696,169","-23,505,788,562","-23,679,484,731"
1,"40,803","-356,100,086",0,"483,900,672",0,0,0,0,0,"-356,100,086",...,0,0,0,0,"7,893,699","-7,893,699","50,476,050","42,582,351","289,331,189","246,748,838"
2,"11,163,888","32,911,139,289","9,136,486","91,348,663,810","10,905","81,488,552",0,"952,971,223","25,911,104","32,834,764,248",...,"405,183","26,316,286","14,105,164",0,"3,258,884,791","-3,246,673,669","4,732,203,581","1,485,529,912","160,287,102,851","158,801,572,939"
3,"13,099,479","103,048,007,398","9,716,714","165,913,534,691","217,819","4,052,730,207",0,"14,314,381,088","1,308,402,254","99,734,162,907",...,"13,683,987","1,322,086,241","247,602,652",0,"7,861,611,339","-6,787,127,750","14,245,875,898","7,458,748,148","330,548,757,847","323,090,009,699"
4,"17,062,882","209,892,996,266","12,933,918","224,427,219,344","512,571","9,267,255,596",0,"78,917,523,426","7,835,773,077","202,264,409,235",...,"35,068,744","7,870,841,821","1,132,603,342",0,"9,625,054,339","-2,886,815,860","29,434,896,806","26,548,080,946","596,985,634,667","570,437,553,721"
5,"16,426,264","308,133,539,048","13,399,370","228,613,974,343","708,277","12,918,147,525",0,"145,308,710,978","15,384,409,414","297,874,695,246",...,"44,439,754","15,428,849,169","2,246,005,407",0,"9,623,768,249","3,559,075,513","43,201,817,396","46,760,892,909","737,545,309,576","690,784,416,667"
6,"32,245,650","934,370,371,811","26,462,074","504,985,007,738","2,576,961","49,741,642,650",0,"517,481,951,945","57,357,452,379","897,078,916,702",...,"167,054,366","57,524,506,746","8,455,833,007",0,"24,435,002,992","24,633,670,747","129,958,917,491","154,592,588,237","1,996,383,921,371","1,841,791,333,133"
7,"23,253,128","1,033,360,877,365","18,687,850","400,277,292,442","3,108,441","68,016,158,081",0,"636,956,189,171","79,458,890,462","984,343,757,059",...,"115,624,242","79,574,514,703","12,110,585,410",0,"19,459,290,885","48,004,638,409","140,331,023,448","188,335,661,857","2,019,630,032,928","1,831,294,371,071"
8,"41,630,877","3,538,453,845,943","28,108,545","663,440,283,642","11,802,357","317,086,841,717",0,"2,617,882,984,037","354,639,365,848","3,320,930,454,641",...,"174,226,745","354,813,592,592","50,022,084,138",0,"16,825,905,328","287,965,603,127","478,304,663,978","766,270,267,105","5,769,908,841,586","5,003,638,574,481"
9,"13,618,878","2,751,636,533,926","5,331,880","131,254,252,528","7,942,273","250,200,391,353",0,"2,368,720,855,161","425,449,380,338","2,579,294,747,573",...,"304,009,361","425,753,389,699","22,445,769,771","1,179,894,187","1,576,754,424","402,910,759,691","306,760,079,281","709,670,838,971","3,708,075,214,598","2,998,404,375,627"


#### By decile

In [14]:
base_dist, ubi_dist = base_calc.distribution_tables(ubi_calc)
ubi_dist_rev, base_dist_rev = ubi_calc.distribution_tables(base_calc)

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ValueError: "Buffer dtype mismatch, expected 'Python object' but got 'long'" in 'pandas._libs.lib.is_bool_array' ignored


Compare baseline calcs.

In [15]:
base_dist

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,989,410","39,623,179,137","13,205,608","184,888,816,799","21,394","250,664,062",0,"1,203,312,633","35,376,583","39,430,895,413",...,"2,049,897","37,426,480","21,099,418",0,"7,274,774,300","-7,258,447,239","8,998,779,111","1,740,331,872","66,885,146,258","65,144,814,386"
1,"16,989,385","163,814,748,833","12,826,534","231,143,100,846","347,950","6,555,240,462",0,"29,103,382,821","2,735,354,665","158,382,909,751",...,"23,958,332","2,759,312,997","719,848,421",0,"15,971,622,216","-13,932,157,640","22,925,388,730","8,993,231,090","302,667,637,932","293,674,406,842"
2,"16,988,538","258,689,773,353","13,615,955","242,932,111,825","538,916","10,154,582,652",0,"90,520,576,000","9,065,484,690","250,273,457,395",...,"37,405,566","9,102,890,256","2,314,851,877",0,"17,662,697,757","-10,874,659,378","36,662,313,579","25,787,654,201","451,379,759,082","425,592,104,881"
3,"16,989,916","363,372,051,152","14,221,602","255,597,429,552","748,553","14,322,723,008",0,"165,310,424,954","17,364,815,909","351,896,968,054",...,"48,721,143","17,413,537,052","4,386,637,352",0,"16,310,341,761","-3,283,442,061","51,440,814,726","48,157,372,664","595,244,511,964","547,087,139,300"
4,"16,989,964","494,034,273,419","14,340,704","271,011,782,927","1,110,724","22,452,845,109",0,"263,620,607,977","28,160,227,360","476,755,680,276",...,"73,418,250","28,233,645,610","7,004,025,255",0,"13,728,852,644","7,500,767,712","69,485,336,932","76,986,104,643","764,040,659,035","687,054,554,392"
5,"16,989,299","664,352,808,984","13,956,021","280,125,859,798","1,829,975","37,693,236,434",0,"400,015,625,047","44,779,924,847","636,298,525,031",...,"79,873,679","44,859,798,526","9,920,588,994",0,"9,371,658,101","25,567,551,430","92,093,295,338","117,660,846,767","976,021,039,258","858,360,192,491"
6,"16,988,353","887,045,801,618","13,246,612","283,165,700,867","2,782,361","63,555,809,383",0,"582,888,556,122","71,228,716,464","841,051,427,033",...,"95,597,740","71,324,314,205","13,180,365,629",0,"5,764,530,465","52,379,418,111","120,446,180,894","172,825,599,005","1,253,156,591,321","1,080,330,992,316"
7,"16,990,524","1,219,318,826,428","12,050,418","271,430,561,396","4,137,030","102,553,006,495",0,"875,974,857,700","111,509,424,651","1,147,432,044,781",...,"90,133,823","111,599,558,474","17,050,132,578",0,"3,402,674,941","91,146,750,955","166,177,263,250","257,324,014,205","1,635,946,426,708","1,378,622,412,503"
8,"16,989,387","1,759,656,241,267","9,765,304","232,700,456,138","6,596,340","181,310,397,656",0,"1,363,168,223,187","191,777,517,456","1,637,674,493,852",...,"74,596,573","191,852,114,030","20,845,572,228",0,"2,019,332,590","168,987,209,212","238,136,585,030","407,123,794,242","2,260,414,652,238","1,853,290,857,996"
9,"16,990,063","4,163,067,699,326","6,844,137","166,639,072,953","9,694,126","311,165,651,236",0,"3,681,716,721,139","747,740,673,998","3,946,961,138,153",...,"1,137,040,949","748,877,714,947","21,347,839,482","8,063,819,946","1,336,104,821","734,257,590,591","386,269,708,312","1,120,527,298,903","4,881,440,692,833","3,760,913,393,930"


In [16]:
base_dist_rev

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,988,107","50,211,164,578","13,451,262","167,353,552,988","60,050","910,438,834",0,"2,180,753,070","114,894,283","49,486,484,895",...,"4,223,408","119,117,691","38,349,394",0,"6,793,436,527","-6,712,668,229","10,283,059,325","3,570,391,096","83,020,309,839","79,449,918,743"
1,"16,990,397","166,526,367,486","12,434,598","217,905,002,384","417,099","7,949,723,522",0,"47,605,259,285","4,580,084,060","159,928,330,545",...,"24,825,747","4,604,909,807","727,491,164",0,"9,705,777,050","-5,828,358,407","23,341,381,688","17,513,023,281","320,335,852,500","302,822,829,219"
2,"16,989,365","271,567,453,082","13,572,969","230,104,960,028","641,199","11,325,683,341",0,"118,074,340,863","12,211,386,820","262,435,523,621",...,"50,999,098","12,262,385,918","1,793,901,430",0,"9,727,068,276","741,416,212","37,976,115,538","38,717,531,750","468,764,123,204","430,046,591,454"
3,"16,989,530","382,034,163,178","13,986,288","248,216,611,615","938,618","17,733,681,604",0,"193,489,963,472","20,756,171,664","368,273,224,140",...,"69,237,947","20,825,409,611","3,141,572,498",0,"11,155,046,432","6,528,790,682","53,498,533,099","60,027,323,780","610,549,298,926","550,521,975,146"
4,"16,989,453","508,395,540,517","13,919,934","268,864,087,058","1,439,063","27,650,367,990",0,"284,118,581,741","31,143,885,880","487,736,564,965",...,"82,835,890","31,226,721,770","4,653,499,177",0,"13,303,605,661","13,269,616,932","70,622,341,284","83,891,958,217","773,764,541,882","689,872,583,666"
5,"16,989,552","661,706,154,839","13,878,164","286,277,801,235","1,930,670","39,957,421,918",0,"394,208,522,069","47,665,566,782","632,547,371,423",...,"69,248,384","47,734,815,166","7,010,344,865",0,"14,618,746,794","26,105,723,507","90,753,979,995","116,859,703,502","976,363,232,873","859,503,529,371"
6,"16,989,302","869,983,069,979","13,400,452","296,918,993,153","2,608,420","59,169,666,102",0,"559,504,051,525","71,203,643,231","828,022,137,671",...,"113,968,314","71,317,611,545","10,845,513,579",0,"12,781,402,455","47,690,695,512","117,711,356,097","165,402,051,609","1,242,990,115,841","1,077,588,064,232"
7,"16,990,054","1,204,366,662,710","12,495,384","292,218,573,181","3,748,173","94,259,478,729",0,"847,649,486,144","108,043,333,084","1,138,487,675,382",...,"50,970,834","108,094,303,918","17,481,329,245",0,"8,376,416,294","82,236,558,379","163,525,630,007","245,762,188,385","1,616,515,347,224","1,370,753,158,839"
8,"16,989,247","1,742,416,114,452","10,233,364","247,301,505,701","6,157,823","172,892,541,390",0,"1,338,053,460,096","185,394,156,896","1,624,835,294,017",...,"69,388,818","185,463,545,714","25,170,338,917",0,"4,299,465,757","155,993,741,040","235,806,069,739","391,799,810,779","2,233,469,804,757","1,841,669,993,979"
9,"16,989,833","4,155,768,712,696","6,700,481","164,473,805,758","9,866,252","318,165,153,067",0,"3,668,637,869,313","743,284,393,923","3,934,404,933,082",...,"1,127,097,512","744,411,491,435","25,928,620,963","8,063,819,946","2,081,624,351","724,465,066,066","389,117,199,129","1,113,582,265,195","4,861,424,489,584","3,747,842,224,388"


Compare UBI distributions.

In [17]:
ubi_dist

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,989,410","39,623,179,137","13,205,608","184,888,816,799","21,394","250,664,062",0,"1,203,312,633","35,376,583","39,430,895,413",...,"2,049,897","37,426,480","21,099,418",0,"7,274,774,300","-7,258,447,239","8,998,779,111","1,740,331,872","290,766,765,858","289,026,433,986"
1,"16,989,385","163,814,748,833","12,826,534","231,143,100,846","347,950","6,555,240,462",0,"29,103,382,821","2,735,354,665","158,382,909,751",...,"23,958,332","2,759,312,997","719,848,421",0,"15,971,622,216","-13,932,157,640","22,925,388,730","8,993,231,090","542,166,753,232","533,173,522,142"
2,"16,988,538","258,689,773,353","13,615,955","242,932,111,825","538,916","10,154,582,652",0,"90,520,576,000","9,065,484,690","250,273,457,395",...,"37,405,566","9,102,890,256","2,314,851,877",0,"17,662,697,757","-10,874,659,378","36,662,313,579","25,787,654,201","714,472,075,482","688,684,421,281"
3,"16,989,916","363,372,051,152","14,221,602","255,597,429,552","748,553","14,322,723,008",0,"165,310,424,954","17,364,815,909","351,896,968,054",...,"48,721,143","17,413,537,052","4,386,637,352",0,"16,310,341,761","-3,283,442,061","51,440,814,726","48,157,372,664","881,111,777,364","832,954,404,700"
4,"16,989,964","494,034,273,419","14,340,704","271,011,782,927","1,110,724","22,452,845,109",0,"263,620,607,977","28,160,227,360","476,755,680,276",...,"73,418,250","28,233,645,610","7,004,025,255",0,"13,728,852,644","7,500,767,712","69,485,336,932","76,986,104,643","1,083,360,876,635","1,006,374,771,992"
5,"16,989,299","664,352,808,984","13,956,021","280,125,859,798","1,829,975","37,693,236,434",0,"400,015,625,047","44,779,924,847","636,298,525,031",...,"79,873,679","44,859,798,526","9,920,588,994",0,"9,371,658,101","25,567,551,430","92,093,295,338","117,660,846,767","1,321,544,886,358","1,203,884,039,591"
6,"16,988,353","887,045,801,618","13,246,612","283,165,700,867","2,782,361","63,555,809,383",0,"582,888,556,122","71,228,716,464","841,051,427,033",...,"95,597,740","71,324,314,205","13,180,365,629",0,"5,764,530,465","52,379,418,111","120,446,180,894","172,825,599,005","1,624,634,589,021","1,451,808,990,016"
7,"16,990,524","1,219,318,826,428","12,050,418","271,430,561,396","4,137,030","102,553,006,495",0,"875,974,857,700","111,509,424,651","1,147,432,044,781",...,"90,133,823","111,599,558,474","17,050,132,578",0,"3,402,674,941","91,146,750,955","166,177,263,250","257,324,014,205","2,037,602,794,908","1,780,278,780,703"
8,"16,989,387","1,759,656,241,267","9,765,304","232,700,456,138","6,596,340","181,310,397,656",0,"1,363,168,223,187","191,777,517,456","1,637,674,493,852",...,"74,596,573","191,852,114,030","20,845,572,228",0,"2,019,332,590","168,987,209,212","238,136,585,030","407,123,794,242","2,696,446,577,038","2,289,322,782,796"
9,"16,990,063","4,163,067,699,326","6,844,137","166,639,072,953","9,694,126","311,165,651,236",0,"3,681,716,721,139","747,740,673,998","3,946,961,138,153",...,"1,137,040,949","748,877,714,947","21,347,839,482","8,063,819,946","1,336,104,821","734,257,590,591","386,269,708,312","1,120,527,298,903","5,341,219,207,033","4,220,691,908,130"


In [18]:
ubi_dist_rev

,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,988,107","50,211,164,578","13,451,262","167,353,552,988","60,050","910,438,834",0,"2,180,753,070","114,894,283","49,486,484,895",...,"4,223,408","119,117,691","38,349,394",0,"6,793,436,527","-6,712,668,229","10,283,059,325","3,570,391,096","263,316,627,839","259,746,236,743"
1,"16,990,397","166,526,367,486","12,434,598","217,905,002,384","417,099","7,949,723,522",0,"47,605,259,285","4,580,084,060","159,928,330,545",...,"24,825,747","4,604,909,807","727,491,164",0,"9,705,777,050","-5,828,358,407","23,341,381,688","17,513,023,281","518,078,602,100","500,565,578,819"
2,"16,989,365","271,567,453,082","13,572,969","230,104,960,028","641,199","11,325,683,341",0,"118,074,340,863","12,211,386,820","262,435,523,621",...,"50,999,098","12,262,385,918","1,793,901,430",0,"9,727,068,276","741,416,212","37,976,115,538","38,717,531,750","689,195,806,804","650,478,275,054"
3,"16,989,530","382,034,163,178","13,986,288","248,216,611,615","938,618","17,733,681,604",0,"193,489,963,472","20,756,171,664","368,273,224,140",...,"69,237,947","20,825,409,611","3,141,572,498",0,"11,155,046,432","6,528,790,682","53,498,533,099","60,027,323,780","869,115,126,626","809,087,802,846"
4,"16,989,453","508,395,540,517","13,919,934","268,864,087,058","1,439,063","27,650,367,990",0,"284,118,581,741","31,143,885,880","487,736,564,965",...,"82,835,890","31,226,721,770","4,653,499,177",0,"13,303,605,661","13,269,616,932","70,622,341,284","83,891,958,217","1,080,367,937,882","996,475,979,666"
5,"16,989,552","661,706,154,839","13,878,164","286,277,801,235","1,930,670","39,957,421,918",0,"394,208,522,069","47,665,566,782","632,547,371,423",...,"69,248,384","47,734,815,166","7,010,344,865",0,"14,618,746,794","26,105,723,507","90,753,979,995","116,859,703,502","1,329,899,624,473","1,213,039,920,971"
6,"16,989,302","869,983,069,979","13,400,452","296,918,993,153","2,608,420","59,169,666,102",0,"559,504,051,525","71,203,643,231","828,022,137,671",...,"113,968,314","71,317,611,545","10,845,513,579",0,"12,781,402,455","47,690,695,512","117,711,356,097","165,402,051,609","1,640,467,848,041","1,475,065,796,432"
7,"16,990,054","1,204,366,662,710","12,495,384","292,218,573,181","3,748,173","94,259,478,729",0,"847,649,486,144","108,043,333,084","1,138,487,675,382",...,"50,970,834","108,094,303,918","17,481,329,245",0,"8,376,416,294","82,236,558,379","163,525,630,007","245,762,188,385","2,061,168,265,424","1,815,406,077,039"
8,"16,989,247","1,742,416,114,452","10,233,364","247,301,505,701","6,157,823","172,892,541,390",0,"1,338,053,460,096","185,394,156,896","1,624,835,294,017",...,"69,388,818","185,463,545,714","25,170,338,917",0,"4,299,465,757","155,993,741,040","235,806,069,739","391,799,810,779","2,721,950,893,857","2,330,151,083,079"
9,"16,989,833","4,155,768,712,696","6,700,481","164,473,805,758","9,866,252","318,165,153,067",0,"3,668,637,869,313","743,284,393,923","3,934,404,933,082",...,"1,127,097,512","744,411,491,435","25,928,620,963","8,063,819,946","2,081,624,351","724,465,066,066","389,117,199,129","1,113,582,265,195","5,359,765,569,884","4,246,183,304,688"


### Difference tables

In [19]:
base_calc.difference_table(ubi_calc, groupby='large_income_bins')

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"1,434,322.13",0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-99.48
1,"12,505,153.05",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,271.66
2,"15,293,224.52",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,92.04
3,"20,287,705.88",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,63.95
4,"19,250,235.67",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,52.49
5,"14,981,031.87",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,46.68
6,"27,918,245.44",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,38.55
7,"18,307,252.32",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,31.06
8,"29,309,678.11",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,22.83
9,"10,607,990.67",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.16


In [20]:
ubi_calc.difference_table(base_calc, groupby='large_income_bins')

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,perc_aftertax,pc_aftertaxinc
0,"110,843.71",0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,8.81
1,"40,802.81",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-206.75
2,"11,163,887.91",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-70.49
3,"13,099,479.23",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-45.67
4,"17,062,881.79",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-36.41
5,"16,426,264.16",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-32.78
6,"32,245,649.83",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-30.87
7,"23,253,128.28",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-28.17
8,"41,630,877.47",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-22.57
9,"14,861,024.47",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-11.18
